In [1]:
import tensorflow as tf
print(tf.__version__)

In [14]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LeakyReLU, Dropout, BatchNormalization,Flatten,Conv2D,Reshape,MaxPool2D, Conv2DTranspose,BatchNormalization
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.applications import VGG16
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, os
import random
import cv2
pathname="../input/vggface-using-tripletloss/celebs/celebs"
dirList=os.listdir(pathname)

ln=len(dirList)

In [2]:
# Load in the data
#mnist = tf.keras.datasets.mnist

#(x_train, y_train), (x_test, y_test) = mnist.load_data()

# map inputs to (-1, +1) for better training
#x_train, x_test = x_train / 255.0 * 2 - 1, x_test / 255.0 * 2 - 1
#print("x_train.shape:", x_train.shape)
#print("x_test.shape:", x_test.shape)

In [3]:
#x_train=np.reshape(x_train,(60000,28,28,1))
#x_test=np.reshape(x_test,(10000,28,28,1))

In [5]:
# Get the generator model
def build_generator():
    
    generator=Sequential()
    generator.add(Conv2DTranspose(512, (4, 4),input_shape=(28,28,512), padding="same",activation=LeakyReLU()))
    generator.add(Conv2DTranspose(512, (4, 4),input_shape=(28,28,512), padding="same",activation=LeakyReLU()))
    
    #generator.add(BatchNormalization())
    
    #generator.add(Conv2DTranspose(512, (6, 6), strides=2, padding="same", activation=LeakyReLU()))
    #generator.add(Conv2DTranspose(512, (6, 6), padding="same", activation=LeakyReLU()))
    
    generator.add(BatchNormalization())
    generator.add(Conv2DTranspose(256, (8, 8), strides=2, padding="same", activation=LeakyReLU()))
    generator.add(Conv2DTranspose(256, (8, 8),  padding="same", activation=LeakyReLU()))


    generator.add(BatchNormalization())

    generator.add(Conv2DTranspose(128, (10, 10), strides=2, padding="same", activation=LeakyReLU()))
    generator.add(Conv2DTranspose(128, (10, 10),  padding="same", activation=LeakyReLU()))


    generator.add(BatchNormalization())
    generator.add(Conv2DTranspose(64, (12, 12), strides=2, padding="same", activation=LeakyReLU()))
    generator.add(Conv2DTranspose(filters=64, kernel_size=(12,12), padding="same", activation=LeakyReLU()))
    generator.add(Conv2DTranspose(filters=64, kernel_size=(12,12), padding="same", activation=LeakyReLU()))

    generator.add(BatchNormalization())
    generator.add(Conv2DTranspose(filters=3,kernel_size=(12, 12), padding="same",activation=LeakyReLU()))
    generator.add(Conv2DTranspose(filters=3,kernel_size=(12, 12), padding="same",activation=LeakyReLU()))
    generator.add(Conv2DTranspose(filters=3,kernel_size=(12, 12), padding="same",activation=LeakyReLU()))
    #generator.summary()
    return generator

In [6]:
decoder = Sequential()
decoder.add(Conv2D(filters=3,kernel_size=(12,12),input_shape=(224,224,3),padding="same", activation=LeakyReLU()))

decoder.add(Conv2D(filters=3,kernel_size=(12,12),padding="same", activation=LeakyReLU()))

decoder.add(BatchNormalization())
decoder.add(Conv2D(filters=64,kernel_size=(12,12),padding="same", activation=LeakyReLU()))
decoder.add(Conv2D(filters=64,kernel_size=(12,12),padding="same", activation=LeakyReLU()))


decoder.add(BatchNormalization())
decoder.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

decoder.add(Conv2D(filters=128, kernel_size=(10,10), padding="same", activation=LeakyReLU()))
decoder.add(Conv2D(filters=128, kernel_size=(10,10), padding="same", activation=LeakyReLU()))


decoder.add(BatchNormalization())
decoder.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
decoder.add(Conv2D(filters=256, kernel_size=(8,8), padding="same", activation=LeakyReLU()))
decoder.add(Conv2D(filters=256, kernel_size=(8,8), padding="same", activation=LeakyReLU()))


decoder.add(BatchNormalization())
decoder.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
decoder.add(Conv2D(filters=512, kernel_size=(6,6), padding="same", activation=LeakyReLU()))
decoder.add(Conv2D(filters=512, kernel_size=(6,6), padding="same", activation=LeakyReLU()))


#decoder.add(BatchNormalization())
#decoder.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
#decoder.add(Conv2D(filters=512, kernel_size=(4,4), padding="same", activation=LeakyReLU()))
#decoder.add(Conv2D(filters=512, kernel_size=(4,4), padding="same", activation=LeakyReLU()))


decoder.add(BatchNormalization())

decoder.summary()

In [7]:
discriminator=Sequential()
discriminator.add(VGG16(include_top=False,input_shape=(224,224,3)))
discriminator.add(Flatten())
discriminator.add(Dense(1,activation="sigmoid"))
discriminator.summary()

In [8]:
# Compile both models in preparation for training


# Build and compile the discriminator

discriminator.compile(
    loss='binary_crossentropy',
    optimizer=Adam(0.0002, 0.5),
    metrics=['accuracy'])

# Build and compile the combined model
generator = build_generator()

# Create an input to represent noise sample from latent space
#z = Input(shape=(latent_dim,))

# Pass noise through generator to get an image
#img = generator(z)

# Make sure only the generator is trained
discriminator.trainable = False


autoencoder=Sequential()
autoencoder.add(decoder)
autoencoder.add(generator)
print("Here")
autoencoder.summary()

# The true output is fake, but we label them real!
fake_pred = Sequential()
fake_pred.add(autoencoder)
fake_pred.add(discriminator)

fake_pred.summary()
# Create the combined model object


# Compile the combined model
fake_pred.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))

In [26]:
# Train the GAN


# Config
batch_size = 64
batch=64
epochs = 30000
sample_period = 200 # every `sample_period` steps generate and save some data


# Create batch labels to use when calling train_on_batch
ones = np.ones(batch_size)
zeros = np.zeros(batch_size)

# Store the losses
d_losses = []
g_losses = []

# Create a folder to store generated images
if not os.path.exists('gan_images'):
  os.makedirs('gan_images')

In [10]:
def randImagePicker(batch):
    img=[]
    for x in range(batch):
        pi=random.randint(0,ln-1)
        pl=os.listdir(pathname+"/"+dirList[pi])
        #print(pl)
        pii=random.randint(0,len(pl)-1)
        faceImg1=cv2.imread(pathname+"/"+dirList[pi]+"/"+pl[pii])
        faceImg1=cv2.resize(faceImg1, (224, 224) ,interpolation = cv2.INTER_NEAREST)
        
        img.append(faceImg1)
    return(np.array(img))
    

In [24]:
# A function to generate a grid of random samples from the generator
# and save them to a file
def sample_images(epoch):
  rows, cols = 5, 5
  real_imgs=randImagePicker(rows*cols)
  imgs = autoencoder.predict(real_imgs)
  fig, axs = plt.subplots(rows, cols)
  idx = 0
  for i in range(rows):
    for j in range(cols):
      axs[i,j].imshow(imgs[idx])
      axs[i,j].axis('off')
      idx += 1
  fig.savefig("%d.png" % epoch)
  plt.show()
  plt.close()

In [ ]:
# Main training loop

for epoch in range(epochs):
  ###########################
  ### Train discriminator ###
  ###########################
  
  

  ###########################
  ###   Getting Images.   ###
  ###########################
  real_imgs=randImagePicker(batch_size)
  #print(real_imgs.shape)
  fake_imgs=autoencoder.predict(real_imgs)
  #print(fake_imgs.shape)
  #for i in fake_imgs:
  #  plt.imshow(i)
  #  plt.show()
  


  # Train the discriminator
  # both loss and accuracy are returned
  d_loss_real, d_acc_real = discriminator.train_on_batch(real_imgs, ones)
  d_loss_fake, d_acc_fake = discriminator.train_on_batch(fake_imgs, zeros)
  d_loss = 0.5 * (d_loss_real + d_loss_fake)
  d_acc  = 0.5 * (d_acc_real + d_acc_fake)
  
  
  #######################
  ### Train generator ###
  #######################
  
  
  g_loss = fake_pred.train_on_batch(real_imgs, ones)
  real_imgs=randImagePicker(batch_size)
  g_loss = fake_pred.train_on_batch(real_imgs, ones)
  
  # Save the losses
  d_losses.append(d_loss)
  g_losses.append(g_loss)
  
  if epoch % 100 == 0:
    print(f"epoch: {epoch+1}/{epochs}, d_loss: {d_loss:.2f}, \
      d_acc: {d_acc:.2f}, g_loss: {g_loss:.2f}")
  
  if epoch % sample_period == 0:
    sample_images(epoch)



In [ ]:
plt.plot(g_losses, label='g_losses')
plt.plot(d_losses, label='d_losses')
plt.legend()

In [ ]:
###Always save what you did forgot few timess!!!!!!!!!!
decoder.save('models/decoder')
generator.save('model/generator')
discriminator.save('model/discriminator')